<a href="https://colab.research.google.com/github/tannisthamaiti/DiffusionModels_DDPM_DDIM/blob/main/GenerateLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount the drive
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
# Replace 'your_folder_name' with the actual name of your folder
folder_path = '/content/drive/MyDrive/DiffusionModels_DDPM_DDIM'

# Check if the folder exists
if os.path.exists(folder_path):
  print(f"Folder '{folder_path}' mounted successfully!")
else:
  print(f"Folder '{folder_path}' not found. Please check the path.")

# Now you can access files within the mounted folder
# For example, to list files in the folder:
os.listdir(folder_path)

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
train_file_path = os.path.join(folder_path, "train_labels.csv")
train_file_path_meta = os.path.join(folder_path, "train_metadata.csv")
labels = pd.read_csv(train_file_path)
metadata = pd.read_csv(train_file_path_meta)
array_2000 =[]

In [ ]:
metadata.head()

In [ ]:
#Check 1 image
filename = "ggv_278.jpg"
image_file_path=os.path.join(folder_path,"train/", filename)
img = Image.open(image_file_path)
image_array = np.array(img)
image_array.shape[0]

In [ ]:
def channel3(img, size):
    empty_3d_array = np.empty((size, size, 3))
    empty_3d_array[:,:,0]=np.array(img)
    empty_3d_array[:,:,1]=np.array(img)
    empty_3d_array[:,:,2]=np.array(img)
    print(empty_3d_array.shape)
    return empty_3d_array


In [ ]:
def channel1(img):
    empty_1d_array = np.empty((16, 16, 1))
    empty_1d_array[:,:,0]=np.array(img)
    return empty_1d_array

In [ ]:
def crop_center(image, new_width, new_height):
    # Get the current dimensions of the image
    width, height = image.size

    # Calculate the coordinates for the crop
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    right = (width + new_width) // 2
    bottom = (height + new_height) // 2

    # Perform the crop
    cropped_image = image.crop((left, top, right, bottom))

    return cropped_image




In [ ]:
def process_images(folder_path):
    image_array = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            size=128
            try:
                img = Image.open(file_path)
                cropped_image = crop_center(img, size, size)  # Crop to 128x128
                image_array.append(channel3(cropped_image, size))
                img.close()
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return image_array

In [ ]:
folder_path_image= os.path.join(folder_path,"train/")
image_array= process_images(folder_path_image)

In [ ]:
file_path_label = "wind_label_3D128X128.npy"
file_path = "wind_3D128X128.npy"
np.save(file_path, image_array)
#np.save(file_path_label, labels_array)

In [ ]:
def action_on_image_change(previous_id, current_id,df, labels_array, image_array):
    print(f"Name changed from {previous_id} to {current_id}")

    # Initialize previous name with None
    #previous_id = None
    # Initialize counter for the number of rows processed for the current ID
    rows_processed_for_current_id = 0
    start_index=0
    folder_path = "train/"

    # Loop over DataFrame rows
    for index, row in df.iterrows():
        current_id = row['Storm ID']

        # Check if the name has changed
        filename = row["Image ID"]+".jpg"
        file_path=os.path.join(folder_path, filename)

        if os.path.exists(file_path):
                #if rows_processed_for_current_id<10:

            try:
                img = Image.open(file_path)
                #resized_img = img.resize((128, 128))
                cropped_image=crop_center(img, 128, 128)
                image_array.append(channel3(cropped_image))

                img.close()
                labels_array.append(action_on_id_change(row["Image ID"] ))
            except:
                print("cannot identify image file",file_path)

        else:
            continue

    return labels_array, image_array

In [ ]:
print(len(labels_array))